In [ ]:
!git clone https://github.com/inttx/DLAM_SealedSurfaces.git
!pip install -r requirements.txt
%cd src

In [ ]:
from torch.utils.data import DataLoader
from torch import nn
from torch.optim import AdamW

from settings import *
from dataset import PotsdamDataset
from model import custom_resnet18
from train import train_loop

In [ ]:
try:
    import os
    from google.colab import drive
    MOUNTPOINT = '/content/dlam_datasets/'
    drive.mount(MOUNTPOINT)

    IMAGE_PATH = os.path.join(MOUNTPOINT, 'My Drive', '2_Ortho_RGB')
    LABEL_PATH = os.path.join(MOUNTPOINT, 'My Drive', '2_Ortho_RGB')
except:
    IMAGE_PATH = '../data/2_Ortho_RGB'
    LABEL_PATH = '../data/5_Labels_all'

In [ ]:
# Hyperparameters
patch_size = 250
stride = 250
batch_size = 512
num_epochs = 2
lr = 0.001

In [ ]:
dataset = PotsdamDataset(IMAGE_PATH, LABEL_PATH, patch_size=patch_size, stride=stride, device=DEVICE)

In [ ]:
model = custom_resnet18(patch_size=patch_size, device=DEVICE)

In [ ]:
dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
loss_fn = nn.MSELoss()
optimizer = AdamW(model.parameters(), lr=lr)

In [ ]:
train_loop(dataloader=dataloader, model=model, loss_fn=loss_fn, optimizer=optimizer, num_epochs=num_epochs, device=DEVICE)